In [86]:
import numpy as np
import torch
import yaml
from models.yolo import Model
from PIL import Image
from skimage import io
#from utils.torch_utils import  ModelEMA, select_device, intersect_dicts

In [87]:
def intersect_dicts(da, db, exclude=()):
    # Dictionary intersection of matching keys and shapes, omitting 'exclude' keys, using da values
    return {k: v for k, v in da.items() if k in db and not any(x in k for x in exclude) and v.shape == db[k].shape}

In [88]:
#model = torch.hub.load('ultralytics/yolov5','yolov5x',classes =5)

In [89]:
weights = "weights/best.pt"
device  = torch.device('cuda')
data_yml = 'data/road.yaml'
hyp_yml = 'data/hyp.scratch.yaml'
with open(data_yml) as f:
    data_dict = yaml.load(f, Loader=yaml.FullLoader)  # data dict
with open(hyp_yml) as f:
    hyp = yaml.load(f, Loader=yaml.FullLoader)  # load hyps
#model.load_state_dict(weights.state_dict())

# Load Model

ckpt = torch.load(weights, map_location=device)  # load checkpoint
if hyp.get('anchors'):
    ckpt['model'].yaml['anchors'] = round(hyp['anchors'])  # force autoanchor
cfg = "yolov5x_road.yaml"

In [90]:
model = Model(cfg, ch=3).to(device)
exclude = ['anchor'] if cfg else []  # exclude keys
state_dict = ckpt['model'].float().state_dict()  # to FP32
state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=exclude)  # intersect
model.load_state_dict(state_dict, strict=False)  # load
#logger.info('Transferred %g/%g items from %s' % (len(state_dict), len(model.state_dict()), weights))  # report



                 from  n    params  module                                  arguments                     
  0                -1  1      8800  models.common.Focus                     [3, 80, 3]                    
  1                -1  1    115520  models.common.Conv                      [80, 160, 3, 2]               
  2                -1  4    315680  models.common.BottleneckCSP             [160, 160, 4]                 
  3                -1  1    461440  models.common.Conv                      [160, 320, 3, 2]              
  4                -1 12   3311680  models.common.BottleneckCSP             [320, 320, 12]                
  5                -1  1   1844480  models.common.Conv                      [320, 640, 3, 2]              
  6                -1 12  13228160  models.common.BottleneckCSP             [640, 640, 12]                
  7                -1  1   7375360  models.common.Conv                      [640, 1280, 3, 2]             
  8                -1  1   4099840  

_IncompatibleKeys(missing_keys=['model.24.anchors'], unexpected_keys=[])

In [91]:
model.eval()
im =io.imread("test.jpg") 
im = np.expand_dims(im,0)
image = torch.from_numpy(im)
print(image.shape)

torch.Size([1, 600, 600, 3])


In [92]:
first_parameter = next(model.parameters())
input_shape = first_parameter.size()
print(input_shape)

torch.Size([80, 12, 3, 3])


In [93]:
inf_out, train_out = model(image)

RuntimeError: Sizes of tensors must match except in dimension 1. Got 2 and 1 in dimension 3 (The offending index is 2)